In [ ]:
import ctypes
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from script.plotDisplayLinear import plotDisplay
from ctypes import c_char_p
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import random

In [ ]:
linear_model = ctypes.CDLL("modele/linear/target/release/liblinear_classification.so")


linear_model.LM_init.argtypes = [
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_char
]
linear_model.LM_init.restype = ctypes.POINTER(ctypes.c_void_p)

linear_model.LM_free.argtypes = [ctypes.POINTER(ctypes.c_void_p)]

linear_model.LM_train.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_size_t,
    ctypes.c_size_t
]

linear_model.LM_predict.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

In [ ]:
# @numba.jit(nopython=True)
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        # img = Image.open(img_path).convert('L')  # Conversion en gris
        img = Image.open(img_path).convert('RGB')
        img = img.resize((32, 32))  # Resize les images
        img_array = np.array(img, dtype=np.float64).flatten()  # Flatten les vecteurs
        images.append(img_array)
        labels.append(label)
    return images, labels

folder_a = 'data/dataset/resized/gas_giant'
folder_b = 'data/dataset/resized/neptune-like'
folder_c = 'data/dataset/resized/super_earth'

images_a, labels_a = load_images_from_folder(folder_a, [1.0, 0.0, 0.0])
images_b, labels_b = load_images_from_folder(folder_b, [0.0, 1.0, 0.0])
images_c, labels_c = load_images_from_folder(folder_c, [0.0, 0.0, 1.0])

X = np.array(images_a + images_b + images_c)
y = np.array(labels_a + labels_b + labels_c)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=None)



In [ ]:
X_train_mean = np.mean(X_train)
X_train_std = np.std(X_train)

print(X_train[5])
print(X_test[:5])

x_train = (X_train - X_train_mean) / X_train_std
x_test = (X_test - X_train_mean) / X_train_std
print("-----------------")
print(X_train[5])
print(X_test[:5])

In [ ]:
# XOR test
learning_rate = 0.05
weights = np.array([0.1, -0.4, 0.6, -0.24], dtype=np.float64, order='C')
bias = 0.2
epochs = 10
activation = b't'
# Initialisation du modèle
model = linear_model.LM_init(learning_rate, weights, weights.size, bias, activation)


n_samples, n_features = x_train.shape[0], x_train.shape[1]

linear_model.LM_train(model, x_train.flatten(), y_train.flatten(), n_samples, n_features, epochs)

predictions = np.zeros(n_samples, dtype=np.float64)

linear_model.LM_predict(
    model,
    x_train,
    n_samples,
    n_features,
    predictions
)

print("---------------")
print(predictions)
print("---------------")

linear_model.LM_free(model)



# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(x_train[:, 0], x_train[:, 1], y_train, color='blue', label='Données réelles')
# ax.scatter(x_train[:, 0], x_train[:, 1], predictions, color='red', label='Prédictions')
# ax.set_xlabel('X1')
# ax.set_ylabel('X2')
# plt.legend()
# plt.show()
# plt.clf()

# Xor

# Linear multi class